In [67]:
Link = 'https://github.com/natsunoyuki/blog_posts/blob/main/data_science/Bayesian%20Optimization%20of%20Model%20Hyperparameters.ipynb'

BASIC

In [68]:
SEED = 1412
Test_Ratio = 0.2
para_split_val = '_01412_'
import os 
import pandas as pd

In [69]:
Home = os.getcwd()
DataFolder = 'Data'
try:
    os.mkdir(DataFolder)
except:
    print('Already Got the Data Folder')
DataFolder = os.path.join(Home,DataFolder)

BayesianResultFolder = 'Bayesian_Result'
try:
    os.mkdir(BayesianResultFolder)
except:
    print('Already Got this Data Folder too')
BayesianResultFolder = os.path.join(Home,BayesianResultFolder)

Energy_result = 'Energy_Result'
try:
    os.mkdir(Energy_result)
except:
    print('High on Energy')
Energy_result = os.path.join(Home,Energy_result)


Already Got the Data Folder
Already Got this Data Folder too
High on Energy


BASIC

DB

In [70]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

def Make_db():
  print("New db")
  X,y = make_classification(n_samples=10000,n_features=5,n_informative=2,n_classes=2,n_clusters_per_class=1,flip_y=float(f'0.{SEED}'),shuffle=False,random_state=SEED)
  #95 -> 4.5%
  #90 -> 14%
  #89 -> 14.12%
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = Test_Ratio, random_state = SEED)

  Train = pd.DataFrame(X_train)
  Train['Answers'] = y_train
  Test = pd.DataFrame(X_test)
  Test['Answers'] = y_test

  Train_Folder = os.path.join(DataFolder,'Train.csv')
  Test_Folder = os.path.join(DataFolder,'Test.csv')

  Train.to_csv(Train_Folder)
  Test.to_csv(Test_Folder)


In [71]:
if 'Train.csv' not in os.listdir(DataFolder) or 'Test.csv' not in os.listdir(DataFolder): 
  Make_db()

  remove_lst = [ os.path.join(BayesianResultFolder,i) for i in os.listdir(BayesianResultFolder)]
  for i in remove_lst:
    os.remove(i)

  remove_lst = [ os.path.join(Energy_result,i) for i in os.listdir(Energy_result)]
  for i in remove_lst:
    os.remove(i)

#New data new results

DB

FILES

In [72]:
Train_File = os.path.join(DataFolder,'Train.csv')
Test_File = os.path.join(DataFolder,'Test.csv')

In [73]:
Train = pd.read_csv(Train_File)
Test = pd.read_csv(Test_File)
col_lst = list(Test.columns)
col_lst.remove('Answers')
col_lst = [i for i in col_lst if not i.startswith('Unnamed')]
X_train = Train[col_lst]
X_test = Test[col_lst]
y_train = Train['Answers']
y_test = Test['Answers']

FILES

In [74]:
len(Test),len(Train)

(2000, 8000)

In [75]:
import json
import numpy as np 
import niapy
from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV
from sklearn.ensemble import RandomForestClassifier


CUCKOO SEARCH

In [76]:
from niapy.algorithms.basic import CuckooSearch

CucKoo_pbounds = {'population_size':(10,100),'pa':(0.1,1.0) ,'n_estimator':(10,1000),'criterion':(0,1),'max_feature':(0,1)}

Cuckoo_df = pd.DataFrame(data={'population_size':[],'Probability_Abandonment':[],'n_estimators':[],'criterion':[],'max_features':[],'Accuracy':[],'Time_Taken (s)':[],'Energy_Consumed (J)':[],'Carbon_Produced (g)':[]})


def Cuckoo_value_update(para_str,val,time_taken,energy_used,carbon):
    temp_para = para_str.split(para_split_val)
    temp_para = temp_para + [val,time_taken,energy_used,carbon]
    Cuckoo_df.loc[len(Cuckoo_df.index)] = temp_para

def Save_Cuckoo_df():
    Cuckoo_df.to_csv(os.path.join(Energy_result,'Cuckoo_Results.csv'))


def mdl_Cuckoo(population_size,pa,n_estimator,criterion,max_feature):
    
    population_size = int(population_size)
    n_estimator = int(n_estimator)
    if criterion < 0.5:
        criterion = 'gini'
    else:
        criterion = 'entropy'
    if max_feature < 0.34:
        max_feature = 'sqrt'
    elif max_feature < 0.67:
        max_feature = 'log2'
    else:
        max_feature = None

    combo_str = f'{population_size}_01412_{pa}_01412_{n_estimator}_01412_{criterion}_01412_{max_feature}'

    Algo = CuckooSearch()
    Algo.set_parameters(population_size=population_size,pa=pa,seed=SEED)

    nia_mdl = NatureInspiredSearchCV(
        estimator=RandomForestClassifier(n_estimators=n_estimator,criterion=criterion,max_features=max_feature),
        param_grid={},
        algorithm=Algo,
        runs=1
    )
    return nia_mdl, combo_str

CUCKOO SEARCH

FIRE FLY 

In [77]:
from niapy.algorithms.basic import FireflyAlgorithm

FireFly_pbounds = {'population_size':(10,100),'alpha':(0.1,1.0),'beta0':(0.01,100),'gamma':(0.1,1.0),'theta':(0.1,1.0),'n_estimator':(10,1000),'criterion':(0,1),'max_feature':(0,1)}

Firefly_df = pd.DataFrame(data={'population_size':[],'alpha':[],'beta0' : [], 'gamma' : [], 'theta' : [], 'n_estimators':[],'criterion':[],'max_features':[],'Accuracy':[],'Time_Taken (s)':[],'Energy_Consumed (J)':[],'Carbon_Produced (g)':[]})


def Firefly_value_update(para_str,val,time_taken,energy_used,carbon):
    temp_para = para_str.split(para_split_val)
    temp_para = temp_para + [val,time_taken,energy_used,carbon]
    Firefly_df.loc[len(Firefly_df.index)] = temp_para

def Save_Firefly_df():
    Firefly_df.to_csv(os.path.join(Energy_result,'Firefly_Results.csv'))


def mdl_Firefly(population_size,alpha,beta0,gamma,theta,n_estimator,criterion,max_feature):
    
    population_size = int(population_size)
    n_estimator = int(n_estimator)
    if criterion < 0.5:
        criterion = 'gini'
    else:
        criterion = 'entropy'
    if max_feature < 0.34:
        max_feature = 'sqrt'
    elif max_feature < 0.67:
        max_feature = 'log2'
    else:
        max_feature = None

    combo_str = f'{population_size}_01412_{alpha}_01412_{beta0}_01412_{gamma}_01412_{theta}_01412_{n_estimator}_01412_{criterion}_01412_{max_feature}'

    Algo = FireflyAlgorithm()
    Algo.set_parameters(population_size=population_size,alpha=alpha,beta0=beta0,gamma=gamma,theta=theta,seed=SEED)

    nia_mdl = NatureInspiredSearchCV(
        estimator=RandomForestClassifier(n_estimators=n_estimator,criterion=criterion,max_features=max_feature),         
        param_grid={},
        algorithm=Algo,
        runs=1
    )
    return nia_mdl, combo_str

FIRE FLY 

BAT

In [78]:
from niapy.algorithms.basic import BatAlgorithm

BAT_pbounds = {'population_size':(10,100),'loudness':(0.1,1.0),'pulse_rate':(0.1,1.0),'gamma':(0.1,1.0),'alpha':(0.1,1.0),'min_frequency':(0,10),'max_frequency':(10,100),'n_estimator':(10,1000),'criterion':(0,1),'max_feature':(0,1)}

Bat_df = pd.DataFrame(data={'population_size':[],'loudness':[],'pulse_rate':[],'gamma':[],'alpha':[],'min_frequency':[],'max_frequency':[],'n_estimators':[],'criterion':[],'max_features':[],'Accuracy':[],'Time_Taken (s)':[],'Energy_Consumed (J)':[],'Carbon_Produced (g)':[]})


def Bat_value_update(para_str,val,time_taken,energy_used,carbon):
    temp_para = para_str.split(para_split_val)
    temp_para = temp_para + [val,time_taken,energy_used,carbon]
    Bat_df.loc[len(Bat_df.index)] = temp_para

def Save_Bat_df():
    Bat_df.to_csv(os.path.join(Energy_result,'Bat_Results.csv'))

def mdl_Bat(population_size,loudness,pulse_rate,gamma,alpha,min_frequency,max_frequency,n_estimator,criterion,max_feature):
    
    population_size = int(population_size)
    n_estimator = int(n_estimator)
    if criterion < 0.5:
        criterion = 'gini'
    else:
        criterion = 'entropy'
    if max_feature < 0.34:
        max_feature = 'sqrt'
    elif max_feature < 0.67:
        max_feature = 'log2'
    else:
        max_feature = None

    combo_str = f'{population_size}_01412_{loudness}_01412_{pulse_rate}_01412_{gamma}_01412_{alpha}_01412_{min_frequency}_01412_{max_frequency}_01412_{n_estimator}_01412_{criterion}_01412_{max_feature}'

    Algo = BatAlgorithm()
    Algo.set_parameters(population_size=population_size,loudness=loudness,pulse_rate=pulse_rate,alpha=alpha,gamma=gamma,min_frequency=min_frequency,max_frequency=max_frequency,seed=SEED)

    nia_mdl = NatureInspiredSearchCV(
        estimator=RandomForestClassifier(n_estimators=n_estimator,criterion=criterion,max_features=max_feature),         
        param_grid={},
        algorithm=Algo,
        runs=1
    )
    return nia_mdl, combo_str

BAT

SELF ADAPTIVE BAT

In [79]:
from niapy.algorithms.modified import AdaptiveBatAlgorithm

SABA_pbounds = {'population_size':(10,100),'loudness':(0.1,1.0),'pulse_rate':(0.1,1.0),'epsilon':(0,0.1),'alpha':(0.1,1.0),'min_frequency':(0,10),'max_frequency':(10,100),'n_estimator':(10,1000),'criterion':(0,1),'max_feature':(0,1)}

SBA_df = pd.DataFrame(data={'population_size':[],'loudness':[],'pulse_rate':[],'epsilon':[],'alpha':[],'min_frequency':[],'max_frequency':[],'n_estimators':[],'criterion':[],'max_features':[],'Accuracy':[],'Time_Taken (s)':[],'Energy_Consumed (J)':[],'Carbon_Produced (g)':[]})


def SBA_value_update(para_str,val,time_taken,energy_used,carbon):
    temp_para = para_str.split(para_split_val)
    temp_para = temp_para + [val,time_taken,energy_used,carbon]
    SBA_df.loc[len(SBA_df.index)] = temp_para

def Save_SBA_df():
    SBA_df.to_csv(os.path.join(Energy_result,'SBA_Results.csv'))

def mdl_SBA(population_size,loudness,pulse_rate,epsilon,alpha,min_frequency,max_frequency,n_estimator,criterion,max_feature):
    
    population_size = int(population_size)
    n_estimator = int(n_estimator)
    if criterion < 0.5:
        criterion = 'gini'
    else:
        criterion = 'entropy'
    if max_feature < 0.34:
        max_feature = 'sqrt'
    elif max_feature < 0.67:
        max_feature = 'log2'
    else:
        max_feature = None

    combo_str = f'{population_size}_01412_{loudness}_01412_{pulse_rate}_01412_{epsilon}_01412_{alpha}_01412_{min_frequency}_01412_{max_frequency}_01412_{n_estimator}_01412_{criterion}_01412_{max_feature}'

    Algo = AdaptiveBatAlgorithm()
    Algo.set_parameters(population_size=population_size,loudness=loudness,pulse_rate=pulse_rate,alpha=alpha,epsilon=epsilon,min_frequency=min_frequency,max_frequency=max_frequency,seed=SEED)

    nia_mdl = NatureInspiredSearchCV(
        estimator=RandomForestClassifier(n_estimators=n_estimator,criterion=criterion,max_features=max_feature),         
        param_grid={},
        algorithm=Algo,
        runs=1
    )
    return nia_mdl, combo_str

SELF ADAPTIVE BAT

PARTICLE SWARM

In [80]:
from niapy.algorithms.basic import ParticleSwarmAlgorithm

PSA_pbounds = {'population_size':(10,100),'c1':(0,4),'c2':(0,4),'w':(0,1),'min_velocity':(-10,0),'max_velocity':(0,10),'repair':(0,5),'n_estimator':(10,1000),'criterion':(0,1),'max_feature':(0,1)}

PSA_df = pd.DataFrame(data={'population_size':[],'c1':[],'c2':[],'w':[],'min_velocity':[],'max_velocity':[],'repair_name':[],'n_estimators':[],'criterion':[],'max_features':[],'Accuracy':[],'Time_Taken (s)':[],'Energy_Consumed (J)':[],'Carbon_Produced (g)':[]})


def PSA_value_update(para_str,val,time_taken,energy_used,carbon):
    temp_para = para_str.split(para_split_val)
    temp_para = temp_para + [val,time_taken,energy_used,carbon]
    PSA_df.loc[len(PSA_df.index)] = temp_para

def Save_PSA_df():
    PSA_df.to_csv(os.path.join(Energy_result,'PSA_Results.csv'))

def mdl_PSA(population_size,c1,c2,w,min_velocity,max_velocity,repair,n_estimator,criterion,max_feature):

    population_size = int(population_size)
    n_estimator = int(n_estimator)
    if criterion < 0.5:
        criterion = 'gini'
    else:
        criterion = 'entropy'
    if max_feature < 0.34:
        max_feature = 'sqrt'
    elif max_feature < 0.67:
        max_feature = 'log2'
    else:
        max_feature = None
    #limit, limit_inverse, wang, rand, reflect
    repair_name = ''
    if repair < 1.0:
        repair = niapy.util.repair.limit
        repair_name = 'limit'
    elif repair < 2.0:
        repair = niapy.util.repair.limit_inverse
        repair_name = 'limit_inverse'
    elif repair < 3.0:
        repair = niapy.util.repair.wang
        repair_name = 'wang'
    elif repair < 4.0:
        repair = niapy.util.repair.rand
        repair_name = 'rand'
    else:
        repair = niapy.util.repair.reflect
        repair_name = 'reflect'


    combo_str = f'{population_size}_01412_{c1}_01412_{c2}_01412_{w}_01412_{min_velocity}_01412_{max_velocity}_01412_{repair_name}_01412_{n_estimator}_01412_{criterion}_01412_{max_feature}'

    Algo = ParticleSwarmAlgorithm()
    Algo.set_parameters(population_size=population_size,c1=c1,c2=c2,w=w,min_velocity=min_velocity,max_velocity=max_velocity,repair=repair,seed=SEED) 

    nia_mdl = NatureInspiredSearchCV(
        estimator=RandomForestClassifier(n_estimators=n_estimator,criterion=criterion,max_features=max_feature),         
        param_grid={},
        algorithm=Algo,
        runs=1,
    )
    return nia_mdl, combo_str

Link = 'https://www.researchgate.net/profile/Mohamed_Mourad_Lafifi/post/Which-is-the-best-swarm-size-in-PSO/attachment/5b5b6f85b53d2f89289c14e1/AS%3A653084896288769%401532718981208/download/Good+Parameters+for+Particle+Swarm+Optimization.pdf'

PARTICLE SWARM

Camel Algorithm

In [81]:
from niapy.algorithms.basic import CamelAlgorithm

camel_bounds = {'population_size':(10,100), 'burden_factor':(0.1,1.0), 'death_rate':(0.1,1.0), 'visibility':(0.1,1.0),'supply_init':(1,1000),'endurance_init':(0,1000), 'min_temperature':(-100,0), 'max_temperature':(1,100),'n_estimator':(10,1000),'criterion':(0,1),'max_feature':(0,1)}

camel_df = pd.DataFrame(data={'population_size':[],'burden_factor':[],'death_rate':[],'visibility':[],'supply_init':[],'endurance_init':[],'min_temperature':[],'max_temperature':[],'n_estimators':[],'criterion':[],'max_features':[],'Accuracy':[],'Time_Taken (s)':[],'Energy_Consumed (J)':[],'Carbon_Produced (g)':[]})


def camel_value_update(para_str,val,time_taken,energy_used,carbon):
    temp_para = para_str.split(para_split_val)
    temp_para = temp_para + [val,time_taken,energy_used,carbon]
    camel_df.loc[len(camel_df.index)] = temp_para

def Save_camel_df():
    camel_df.to_csv(os.path.join(Energy_result,'Camel_Result.csv'))

def mdl_camel(population_size,burden_factor,death_rate,visibility,supply_init,endurance_init,min_temperature,max_temperature,n_estimator,criterion,max_feature):

    population_size = int(population_size)
    n_estimator = int(n_estimator)
    if criterion < 0.5:
        criterion = 'gini'
    else:
        criterion = 'entropy'
    if max_feature < 0.34:
        max_feature = 'sqrt'
    elif max_feature < 0.67:
        max_feature = 'log2'
    else:
        max_feature = None

    combo_str = f'{population_size}_01412_{burden_factor}_01412_{death_rate}_01412_{visibility}_01412_{supply_init}_01412_{endurance_init}_01412_{min_temperature}_01412_{max_temperature}_01412_{n_estimator}_01412_{criterion}_01412_{max_feature}'

    Algo = CamelAlgorithm()
    Algo.set_parameters(population_size=population_size,burden_factor=burden_factor,death_rate=death_rate,visibility=visibility,supply_init=supply_init,endurance_init=endurance_init,min_temperature=min_temperature,max_temperature=max_temperature,seed=SEED) 

    nia_mdl = NatureInspiredSearchCV(
        estimator=RandomForestClassifier(n_estimators=n_estimator,criterion=criterion,max_features=max_feature),         
        param_grid={},
        algorithm=Algo,
        runs=1,
    )
    return nia_mdl, combo_str

Camel Algorithm

BAYESIAN PARAMETERS

In [82]:

NIA_Name = {0:'Cuckoo Search',1:'Fire Fly',2:'Bat',3:'Self Adaptive Bat',4:'Particle Swarm', 5:'Camel Algorithm'}

NIA_pbounds_lst = [CucKoo_pbounds, FireFly_pbounds, BAT_pbounds, SABA_pbounds, PSA_pbounds, camel_bounds]

save_fxn = [Save_Cuckoo_df,Save_Firefly_df,Save_Bat_df,Save_SBA_df,Save_PSA_df,Save_camel_df]

update_fxn = [Cuckoo_value_update,Firefly_value_update,Bat_value_update,SBA_value_update,PSA_value_update,camel_value_update]
#camel_value_update(para_str,val,time_taken,carbon)
mdl_fxn = [mdl_Cuckoo,mdl_Firefly,mdl_Bat,mdl_SBA,mdl_PSA,mdl_camel]
#mdl_camel(para_str)

BAYESIAN PARAMETERS

FUNCTION LIST

In [83]:
from bayes_opt import BayesianOptimization, UtilityFunction
from datetime import datetime as dt 

ENERGY FUNCTIONS

In [84]:
def Get_hms(val):
 
    sec = val/1000
    mini = sec/60 
    hr = mini/60
    ms = sec%1000
    hr %= 24
    mini %= 60
    sec %= 60 

    return int(hr), int(mini), int(sec) 
    

In [85]:
def Check_Time(df_timestamp,h,m,s):
    df_h,df_m,df_s = Get_hms(df_timestamp)
    #print(f'checking {df_h}:{df_m}:{df_s}')
    if df_h == h and df_m == m and df_s == s:
        return True
    return False


In [86]:
def Get_req_start_Idx(s_h,s_m,s_s,prev_idx,flag=True):
    df = pd.read_csv(energy_file)
    for i in range(prev_idx,len(df)):
        df_timestamp = df['TimeStamp (ms)'][i]
        if Check_Time(df_timestamp,s_h,s_m,s_s):
            return i
    if flag:
        print('re try')
        print(prev_idx)
        print(Get_hms(df['TimeStamp (ms)'][len(df)]))
        return Get_req_start_Idx(s_h,s_m,s_s,prev_idx=0,flag=False)
    else:
        return -1


In [87]:
def Cal_Energy(start_idx,e_h,e_m,e_s):
    #total_energy_lst = []
    total_energy = 0
    cpu_e = 0
    monitor_e = 0
    disk_e = 0
    base_e = 0
    df = pd.read_csv(energy_file)
#['TimeStamp (ms)', ' Total Power (W)', ' CPU (W)', ' Monitor (W)',' Disk (W)', ' Base (W)', ' Application (W)']
    for i in range(start_idx,len(df)):
        #total_energy_lst.append(df[' Total Power (W)'][i])
        total_energy += df[' Total Power (W)'][i]
        cpu_e += df[' CPU (W)'][i]
        monitor_e += df[' Monitor (W)'][i]
        disk_e += df[' Disk (W)'][i]
        base_e += df[' Base (W)'][i]
        df_timestamp = df['TimeStamp (ms)'][i]
        if Check_Time(df_timestamp,e_h,e_m,e_s):
            return total_energy,cpu_e,monitor_e,disk_e,base_e, i
    return total_energy,cpu_e,monitor_e,disk_e,base_e, len(df)

In [88]:
def Get_Time_Energy(start_time,end_time,prev_idx=0):

    s_d = start_time.day
    s_h = start_time.hour
    s_m = start_time.minute
    s_s = start_time.second
    #print('Start',s_d,s_h,s_m,s_s)
    e_d = end_time.day 
    e_h = end_time.hour
    e_m = end_time.minute
    e_s = end_time.second
    #print('End',e_d,e_h,e_m,e_s)
    time_taken = 0
    time_taken = 24*(time_taken + e_d-s_d)
    time_taken = 60*(time_taken + e_h-s_h)
    time_taken = 60*(time_taken + e_m - s_m)
    time_taken = (time_taken + e_s - s_s)

    start_idx = Get_req_start_Idx(s_h=s_h,s_m=s_m,s_s=s_s,prev_idx=prev_idx,flag=True)
    if start_idx == -1:
        print("FAILED TO FIND START IDX")
        return
    # total_energy,cpu_e,monitor_e,disk_e, i, df[' Base (W)'][0]
    total,cpu,monitor,disk,base,end_idx = Cal_Energy(start_idx,e_h,e_m,e_s)

    return total,cpu,monitor,disk,base, time_taken, end_idx


ENERGY FUNCTIONS

In [89]:
energy_file = [ os.path.join(os.getcwd(),x) for x in os.listdir() if x.endswith('.csv')][0]
energy_file
#always check for latest copy whenever you check

'c:\\Users\\chand\\Desktop\\temp_here\\Green-Computing\\my_tries\\full\\Kid_The_Energy_Keeper.csv'

BAYESIAN FUNCTION

In [95]:
from matplotlib import pyplot as plt 
def Optimize_and_plot(bounds,curr_mdl_fxn,Algo_name:str,Iters:int,curr_save_fxn,curr_update_fxn,My_prev_idx=0):
    
    print(f'Currently at {Algo_name}')

    optimizer = BayesianOptimization(f = None, pbounds = bounds, verbose = 2, random_state = SEED)
    utility = UtilityFunction(kind = "ucb", kappa = 1.96, xi = 0.01)

    temp_df = pd.DataFrame(data={'para_str':[],'total (J)':[],'cpu (J)':[],'monitor (J)':[],'disk (J)':[],'base (J)':[],'Accuracy':[]})

    for i in range(Iters):
        # Get optimizer to suggest a new parameter value to try.
        next_point = optimizer.suggest(utility)  
        # Evaluate the output of the black_box_function using the new parameter value.
        curr_mdl, para_str = curr_mdl_fxn(**next_point)

        start_time = dt.now()
        curr_mdl.fit(X_train,y_train)
        target = curr_mdl.score(X_test,y_test)
        end_time = dt.now()

        total,cpu,monitor,disk,base, time_taken, My_prev_idx = Get_Time_Energy(start_time,end_time,prev_idx=My_prev_idx)
        '''emission 
        1 kW-hr = 0.85 Kg of CO2 emission 
        36 e5 - 85 e4 mg
        360 - 85
        72 J - 17mg CO2 
        1 J = (17.0/72.0) mg CO2
        ''' 
        curr_update_fxn(para_str,target,time_taken,total,total*(17.0/72.0))
        #(para_str,val,time_taken,energy,carbon)

        temp_df.loc[len(temp_df.index)] = [para_str,total,cpu,monitor,disk,base,target]

        try:
            # Update the optimizer with the evaluation results. This needs to be in try-except
            # to prevent repeat errors from occuring.
            optimizer.register(params = next_point, target = target)
        except:
            print('What was that?')
            pass

    curr_save_fxn()
    temp_df.to_csv(os.path.join(Energy_result,f'{Algo_name} Extra.csv'))    

    plt.plot(range(1, 1+len(optimizer.space.target)), optimizer.space.target, "-o")
    plt.grid(True)
    plt.xlabel("Iteration")
    plt.ylabel("Score")
    plt.show()
    plt.savefig(os.path.join(BayesianResultFolder,f'{Algo_name}_{Iters}_Iterations.svg')  ,format='svg')
    plt.show()

    #os.remove(energy_file)
    #To make sure the file doesn't get like 10GB
    #Na, it removed the col heads as well
    return My_prev_idx 



BAYESIAN FUNCTION

In [96]:
#(bounds,curr_mdl_fxn,Algo_name:str,Iters:int,curr_save_fxn,curr_update_fxn):
My_prev_idx=0
for i in range(1,len(mdl_fxn)):
    My_prev_idx=Optimize_and_plot(bounds=NIA_pbounds_lst[i],curr_mdl_fxn=mdl_fxn[i],Algo_name=NIA_Name[i],curr_save_fxn=save_fxn[i],Iters=100,curr_update_fxn=update_fxn[i],My_prev_idx=My_prev_idx)
    

Currently at Fire Fly
Fitting at most 1.0 candidates
Optimization finished, 1 candidates were fitted
Fitting at most 1.0 candidates
Optimization finished, 1 candidates were fitted
Fitting at most 1.0 candidates
Optimization finished, 1 candidates were fitted
Fitting at most 1.0 candidates
Optimization finished, 1 candidates were fitted
Fitting at most 1.0 candidates
Optimization finished, 1 candidates were fitted
Fitting at most 1.0 candidates
Optimization finished, 1 candidates were fitted
Fitting at most 1.0 candidates
Optimization finished, 1 candidates were fitted
Fitting at most 1.0 candidates
Optimization finished, 1 candidates were fitted
Fitting at most 1.0 candidates
Optimization finished, 1 candidates were fitted
Fitting at most 1.0 candidates
Optimization finished, 1 candidates were fitted
Fitting at most 1.0 candidates
Optimization finished, 1 candidates were fitted
Fitting at most 1.0 candidates
Optimization finished, 1 candidates were fitted
Fitting at most 1.0 candidates

KeyError: 5863

BASE CASE

In [ ]:
curr_mdl = RandomForestClassifier()
start_time = dt.now()
curr_mdl.fit(X_train,y_train)
val = curr_mdl.score(X_test,y_test)
end_time = dt.now()

total,cpu,monitor,disk,base, time_taken, My_prev_idx = Get_Time_Energy(start_time,end_time,prev_idx=My_prev_idx)


In [ ]:
data_dict = {'Para' : ['default'], 'Accuracy' : [val],'Time Taken (s)' :[time_taken], 'Energy Consumes (J)':[total], 'CO2 Emission (mg)' :total*(17.0/72.0)}
df = pd.DataFrame(data=data_dict)
df.to_csv(os.path.join(Energy_result,'Base_Result.csv'))

BASE CASE

function ClickConnect(){
               
console.log("Working"); 
document.querySelector("#comments > span").click() 
}
setInterval(ClickConnect,5000)